#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

The default model for below embedding is `llama2`. So we need to pull the model first with `ollama pull llama2` or `ollama run llama2`, but since the model is too large (around 4 Gig) I use previously downloaded model `gemma:2b`.

In [5]:
embeddings = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs, embeddings)
db

#### Querying

In [6]:
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs

[Document(id='224fe31d-6ea8-4827-9dcb-13112a223d1f', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='0913ac40-a21a-4603-938c-1e3fd637e57f', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the

In [7]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [9]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs

[Document(id='224fe31d-6ea8-4827-9dcb-13112a223d1f', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='0913ac40-a21a-4603-938c-1e3fd637e57f', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the

In [10]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance (Manhattan distance). Therefore, a lower score is better.

In [11]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='224fe31d-6ea8-4827-9dcb-13112a223d1f', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2918.1492)),
 (Document(id='0913ac40-a21a-4603-938c-1e3fd637e57f', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and every

2918.1492 is the least value, so that sentence is the most similar sentence to our query sentence.

In [12]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.287360817193985,
 -2.1569526195526123,
 0.27405163645744324,
 0.9851855635643005,
 0.6354705095291138,
 0.5475373268127441,
 -0.9308937788009644,
 0.10366044193506241,
 -0.0986175611615181,
 -0.7731931209564209,
 1.101796269416809,
 -0.04323110356926918,
 -1.0992375612258911,
 1.102561593055725,
 0.0691760703921318,
 -0.9817805290222168,
 3.061448097229004,
 1.7174644470214844,
 1.1580605506896973,
 0.7883134484291077,
 0.32656005024909973,
 -0.28865623474121094,
 0.3018942177295685,
 1.3419572114944458,
 0.19805380702018738,
 -0.38755175471305847,
 -1.378153920173645,
 -1.2767211198806763,
 -0.5441585183143616,
 -2.1179068088531494,
 -0.20746687054634094,
 -1.4256168603897095,
 1.2182226181030273,
 -0.941696286201477,
 -0.3953571021556854,
 -0.2869212031364441,
 1.8598254919052124,
 0.5384001731872559,
 0.15763400495052338,
 -0.5634590983390808,
 0.3750627040863037,
 0.48169565200805664,
 0.9787423014640808,
 -1.2858556509017944,
 -1.354517936706543,
 0.6358289122581482,
 0.0456117

In [13]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='224fe31d-6ea8-4827-9dcb-13112a223d1f', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='0913ac40-a21a-4603-938c-1e3fd637e57f', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the

#### Saving and Loading

In [15]:
db.save_local("faiss_index")

#### Load the folder

In [19]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
docs = new_db.similarity_search(query)

In [20]:
docs

[Document(id='224fe31d-6ea8-4827-9dcb-13112a223d1f', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='0913ac40-a21a-4603-938c-1e3fd637e57f', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the